In [64]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 23 2015

@author: Anton at 0x0af@ukr.net
"""

import numpy,scipy
import copy, math
from scipy.fftpack import dct, idct
import pywt
from PIL import Image

In [65]:
def DhanalakshmiEmbed(GrayscaleContainerPath,GrayscalePrimaryWatermarkPath,GrayscaleSecondaryWatermarkPath,WatermarkedImagePath,Alpha1,Alpha2):
    """    
    Dhanalakshmi embedding method implementation. 
    
    Outputs the resulting watermarked image
    
    23-July-2015
    """
    
    grayscaleContainer2DArray = numpy.asarray(Image.open(GrayscaleContainerPath).convert("L"))
    grayscalePrimaryWatermark2DArray = numpy.asarray(Image.open(GrayscalePrimaryWatermarkPath).convert("L"))
    grayscaleSecondaryWatermark2DArray = numpy.asarray(Image.open(GrayscaleSecondaryWatermarkPath).convert("L"))
    
    #print grayscaleContainer2DArray.shape
    
    # Perform DWT on PrimaryWatermark
    
    GPWLL,(GPWLH,GPWHL,GPWHH) = pywt.dwt2(grayscalePrimaryWatermark2DArray, 'haar')
    
    # Perform SVD on PrimaryWatermark DWT coeffs
    
    GPWLLU,GPWLLS,GPWLLV = numpy.linalg.svd(GPWLL,full_matrices = True)
    GPWLHU,GPWLHS,GPWLHV = numpy.linalg.svd(GPWLH,full_matrices = True)
    GPWHLU,GPWHLS,GPWHLV = numpy.linalg.svd(GPWHL,full_matrices = True)
    GPWHHU,GPWHHS,GPWHHV = numpy.linalg.svd(GPWHH,full_matrices = True)
    
    #print GPWLLS.shape,GPWLHS.shape,GPWHLS.shape,GPWHHS.shape
    
    # Perform SVD on SecondaryWatermark
    
    GSWU,GSWS,GSWV = numpy.linalg.svd(grayscaleSecondaryWatermark2DArray,full_matrices = True)
    
    #print GSWS.shape
    
    # Alpha-blend SecondaryWatermark singular values into PrimaryWatermark DWT coeffs singular values
    
    GPWLLS += Alpha1 * GSWS
    GPWLHS += Alpha1 * GSWS
    GPWHLS += Alpha1 * GSWS
    GPWHHS += Alpha1 * GSWS
    
    # Perform LogisticMapChaoticEncryption on PrimaryWatermark DWT coeffs singular values
    
    EPWLLS,EPWLHS,EPWHLS,EPWHHS = LogisticMapChaoticEncryption(GPWLLS,GPWLHS,GPWHLS,GPWHHS)
    
    # Break GrayscaleContainer into non-overlapping blocks of PrimaryWatermark's shape an run for each
    
    watermarkedImage2DArray = numpy.copy(grayscaleContainer2DArray)
    
    BLOCK = grayscalePrimaryWatermark2DArray.shape[0]
    COLS = int(math.floor(grayscaleContainer2DArray.shape[0] / grayscalePrimaryWatermark2DArray.shape[0]))
    ROWS = int(math.floor(grayscaleContainer2DArray.shape[1] / grayscalePrimaryWatermark2DArray.shape[1]))
    
    for row in range(1,ROWS+1):
        for col in range(1,COLS+1):

            # Perform DWT on GrayscaleContainer's block

            GCBLL,(GCBLH,GCBHL,GCBHH) = pywt.dwt2(grayscaleContainer2DArray[(row-1)*BLOCK:row*BLOCK,(col-1)*BLOCK:col*BLOCK], 'haar')

            # Perform SVD on GrayscaleContainer's block DWT coeffs

            GCBLLU,GCBLLS,GCBLLV = numpy.linalg.svd(GCBLL,full_matrices = True)
            GCBLHU,GCBLHS,GCBLHV = numpy.linalg.svd(GCBLH,full_matrices = True)
            GCBHLU,GCBHLS,GCBHLV = numpy.linalg.svd(GCBHL,full_matrices = True)
            GCBHHU,GCBHHS,GCBHHV = numpy.linalg.svd(GCBHH,full_matrices = True)

            # Alpha-blend EncryptedPrimaryWatermark singular values into GrayscaleContainer's block DWT coeffs singular values

            GCBLLS += Alpha2 * EPWLLS
            GCBLHS += Alpha2 * EPWLHS
            GCBHLS += Alpha2 * EPWHLS
            GCBHHS += Alpha2 * EPWHHS

            # Perform ISVD to get the WatermarkedImage's block DWT coeffs

            GCBLL  = numpy.dot(GCBLLU, numpy.dot(numpy.diag(GCBLLS), GCBLLV))
            GCBLH  = numpy.dot(GCBLHU, numpy.dot(numpy.diag(GCBLHS), GCBLHV))
            GCBHL  = numpy.dot(GCBHLU, numpy.dot(numpy.diag(GCBHLS), GCBHLV))
            GCBHH  = numpy.dot(GCBHHU, numpy.dot(numpy.diag(GCBHHS), GCBHHV))
            
            # Perform IDWT to get the WatermarkedImage's block
            
            WatermarkedImageBlock = pywt.idwt2((GCBLL,(GCBLH,GCBHL,GCBHH)),'haar')

            # Put WatermarkedImage's block into WatermarkedImage
            
            watermarkedImage2DArray[(row-1)*BLOCK:row*BLOCK,(col-1)*BLOCK:col*BLOCK] = WatermarkedImageBlock
    
    watermarkedImage2DArray[watermarkedImage2DArray>255] = 255
    watermarkedImage2DArray[watermarkedImage2DArray<0] = 0
    
    watermarkedImage = Image.fromarray(numpy.uint8(watermarkedImage2DArray))
    watermarkedImage.show()
    
    # Write image to file
    
    watermarkedImage.save(WatermarkedImagePath)
    
    return

In [66]:
def DhanalakshmiExtract(GrayscaleStegoPath,GrayscaleContainerPath,GrayscalePrimaryWatermarkPath,GrayscaleSecondaryWatermarkPath,GrayscalePrimaryExtractedWatermarkPath,GrayscaleSecondaryExtractedWatermarkPath,Alpha1,Alpha2):
    """
    Dhanalakshmi extracting method implementation.
    
    Outputs the extracted watermark
    
    23-July-2015
    """
    
    grayscaleStego2DArray = numpy.asarray(Image.open(GrayscaleStegoPath).convert("L"))
    grayscaleContainer2DArray = numpy.asarray(Image.open(GrayscaleContainerPath).convert("L"))
    grayscalePrimaryWatermark2DArray = numpy.asarray(Image.open(GrayscalePrimaryWatermarkPath).convert("L"))
    grayscaleSecondaryWatermark2DArray = numpy.asarray(Image.open(GrayscaleSecondaryWatermarkPath).convert("L"))
    
    # TODO: average through blocks for more robustness
    
    BLOCK = grayscalePrimaryWatermark2DArray.shape[0]
    COLS = int(math.floor(grayscaleContainer2DArray.shape[0] / grayscalePrimaryWatermark2DArray.shape[0]))
    ROWS = int(math.floor(grayscaleContainer2DArray.shape[1] / grayscalePrimaryWatermark2DArray.shape[1]))
    
    row = int(ROWS/2+1)
    col = int(COLS/2+1)
    
    # Perform DWT on GrayscaleStego's block
    
    GSBLL,(GSBLH,GSBHL,GSBHH) = pywt.dwt2(grayscaleStego2DArray[(row-1)*BLOCK:row*BLOCK,(col-1)*BLOCK:col*BLOCK], 'haar')
    
    # Perform SVD on GrayscaleStego's block DWT coeffs
    
    GSBLLU,GSBLLS,GSBLLV = numpy.linalg.svd(GSBLL,full_matrices = True)
    GSBLHU,GSBLHS,GSBLHV = numpy.linalg.svd(GSBLH,full_matrices = True)
    GSBHLU,GSBHLS,GSBHLV = numpy.linalg.svd(GSBHL,full_matrices = True)
    GSBHHU,GSBHHS,GSBHHV = numpy.linalg.svd(GSBHH,full_matrices = True)
    
    # Perform DWT on GrayscaleContainer's block
    
    GCBLL,(GCBLH,GCBHL,GCBHH) = pywt.dwt2(grayscaleContainer2DArray[(row-1)*BLOCK:row*BLOCK,(col-1)*BLOCK:col*BLOCK], 'haar')

    # Perform SVD on GrayscaleContainer's block DWT coeffs

    GCBLLU,GCBLLS,GCBLLV = numpy.linalg.svd(GCBLL,full_matrices = True)
    GCBLHU,GCBLHS,GCBLHV = numpy.linalg.svd(GCBLH,full_matrices = True)
    GCBHLU,GCBHLS,GCBHLV = numpy.linalg.svd(GCBHL,full_matrices = True)
    GCBHHU,GCBHHS,GCBHHV = numpy.linalg.svd(GCBHH,full_matrices = True)
            
    # Extract EncryptedPrimaryWatermark DWT coeffs singular values with Alpha2
    
    EPWLLS = (GSBLLS - GCBLLS) / Alpha2
    EPWLHS = (GSBLHS - GCBLHS) / Alpha2
    EPWHLS = (GSBHLS - GCBHLS) / Alpha2
    EPWHHS = (GSBHHS - GCBHHS) / Alpha2
    
    # Perform LogisticMapChaoticDecryption on EncryptedPrimaryWatermark DWT coeffs singular values
    
    DPWLLS,DPWLHS,DPWHLS,DPWHHS = LogisticMapChaoticDecryption(EPWLLS,EPWLHS,EPWHLS,EPWHHS)
    
    # Perform DWT on PrimaryWatermark (non-embedded)
    
    GPWLL,(GPWLH,GPWHL,GPWHH) = pywt.dwt2(grayscalePrimaryWatermark2DArray, 'haar')
    
    # Perform SVD on PrimaryWatermark (non-embedded) DWT coeffs
    
    GPWLLU,GPWLLS,GPWLLV = numpy.linalg.svd(GPWLL,full_matrices = True)
    GPWLHU,GPWLHS,GPWLHV = numpy.linalg.svd(GPWLH,full_matrices = True)
    GPWHLU,GPWHLS,GPWHLV = numpy.linalg.svd(GPWHL,full_matrices = True)
    GPWHHU,GPWHHS,GPWHHV = numpy.linalg.svd(GPWHH,full_matrices = True)
    
    #print GPWLLS.shape,GPWLHS.shape,GPWHLS.shape,GPWHHS.shape
    #print DPWLLS.shape,DPWLHS.shape,DPWHLS.shape,DPWHHS.shape
    
    # Perform ISVD on DecryptedPrimaryWatermark DWT coeffs singular values
    
    DPWLL = numpy.dot(GPWLLU, numpy.dot(numpy.diag(DPWLLS), GPWLLV))
    DPWLH = numpy.dot(GPWLHU, numpy.dot(numpy.diag(DPWLHS), GPWLHV))
    DPWHL = numpy.dot(GPWHLU, numpy.dot(numpy.diag(DPWHLS), GPWHLV))
    DPWHH = numpy.dot(GPWHHU, numpy.dot(numpy.diag(DPWHHS), GPWHHV))
    
    # Perform IDWT on DecryptedPrimaryWatermark DWT coeffs
    
    DecryptedPrimaryWatermark2DArray = pywt.idwt2((DPWLL,(DPWLH,DPWHL,DPWHH)),'haar')
    
    DecryptedPrimaryWatermark2DArray[DecryptedPrimaryWatermark2DArray>255] = 255
    DecryptedPrimaryWatermark2DArray[DecryptedPrimaryWatermark2DArray<0] = 0
    
    DecryptedPrimaryWatermark = Image.fromarray(numpy.uint8(DecryptedPrimaryWatermark2DArray))
    DecryptedPrimaryWatermark.show()
    
    # Write DecryptedPrimaryWatermark to file
    
    DecryptedPrimaryWatermark.save(GrayscalePrimaryExtractedWatermarkPath)
    
    # Extract SecondaryWatermark singular values with Alpha1
    # TODO: extract from all levels and average for more robustness
    
    SWS = (DPWLLS - GPWLLS) / Alpha1
    
    # Perform SVD on SecondaryWatermark (non-embedded)
    
    GSWU,GSWS,GSWV = numpy.linalg.svd(grayscaleSecondaryWatermark2DArray,full_matrices = True)
    
    # Perform ISVD to get SecondaryWatermark
    
    SW2DArray = numpy.dot(GSWU, numpy.dot(numpy.diag(SWS), GSWV))
    
    SW2DArray[SW2DArray>255] = 255
    SW2DArray[SW2DArray<0] = 0
    
    SecondaryWatermark = Image.fromarray(numpy.uint8(SW2DArray))
    SecondaryWatermark.show()
    
    # Write DecryptedPrimaryWatermark to file
    
    SecondaryWatermark.save(GrayscaleSecondaryExtractedWatermarkPath)
    
    return

In [67]:
def LogisticMapChaoticEncryption(LL,HL,LH,HH):
    """
    Logistic map chaotic encryption implementation.
    
    Outputs encrypted watermark
    
    23-July-2015
    """
    
    #TODO: implement logistic map chaotic encryption
    
    return LL,HL,LH,HH #some key?

In [68]:
def LogisticMapChaoticDecryption(LL,HL,LH,HH): #some key?
    """
    Logistic map chaotic decryption implementation.
    
    Outputs original watermark
    
    23-July-2015
    """
    
    #TODO: implement logistic map chaotic decryption
    
    return LL,HL,LH,HH

In [69]:
DhanalakshmiEmbed("C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleContainer.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleWatermark.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleSecondaryWatermark.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\Dhanalakshmi_Watermarked_Image.bmp",0.001,0.001)

In [70]:
DhanalakshmiExtract("C:\Users\Tony\Documents\GitHub\steganography-embedding\Dhanalakshmi_Watermarked_Image.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleContainer.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleWatermark.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleSecondaryWatermark.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\Dhanalakshmi_Extracted_Primary_Watermark.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\Dhanalakshmi_Extracted_Secondary_Watermark.bmp",0.001,0.001)